## export env
export $(cat .env | xargs)

In [10]:
import os
from pymongo.mongo_client import MongoClient

MONGODB_CONNECTION_STRING = os.environ['MONGODB_CONNECTION_STRING']
MONGODB_DATABASE = os.environ['MONGODB_DATABASE']
MONGODB_RAW_COLLECTION = os.environ['MONGODB_RAW_COLLECTION']

MONGODB_RPROCESSED_COLLECTION = os.environ['MONGODB_PROCESSED_COLLECTION']
#connect to connection
client = MongoClient(MONGODB_CONNECTION_STRING)
db = client[MONGODB_DATABASE]
collection = db[MONGODB_RAW_COLLECTION]
print("Connecting to MongoDB", MONGODB_DATABASE, MONGODB_RAW_COLLECTION)

#find one document
document = collection.find({})
data = list(document)
print(len(data))

Connecting to MongoDB SGPropertyHub raw
34759


In [48]:
from pydantic import BaseModel

class DataNew(BaseModel):
    raw_id: str
    category: str
    region: str
    ward: str
    list_time: str
    description: str
    floors: int=None

test = DataNew(data[0])

TypeError: BaseModel.__init__() takes 1 positional argument but 2 were given

In [53]:
import pandas as pd
import json
import hashlib

def add_link_key(item):
    area_str = ''
    list_id_str = ''
    try:
        area_str = item['area_name'].replace(" ", "-")
        list_id_str = str(item['list_id'])
    except:
        pass
 
    url = 'https://www.nhatot.com/mua-ban-nha-dat-{area}-tp-ho-chi-minh/{list_id}.htm'
    item['url'] = url.format(area = area_str, list_id = list_id_str)
    item['id'] = hashlib.sha1(url.encode('utf-8')).hexdigest()

def data_test(item,key_name,value_name):
        new_item = {}
        new_item['raw_id'] = str(item['_id'])
        for i in range(len(key_name)):
            try:
                new_item[key_name[i]]=item[value_name[i]]
            except:
                continue
        return new_item

data_new= []
key_name=['category','region','ward','list_time','description','floors']
value_name=['category_name','region_name','ward_name','list_time','body','floors']
for i in data:
    temp = data_test(i, key_name, value_name)
    data_new.append(temp)


print(data_new[1])

{'raw_id': '64f5ede00459b50fd4b6b9b0', 'category': 'Nhà ở', 'region': 'Tp Hồ Chí Minh', 'ward': 'Phường Cô Giang', 'list_time': 1693458004719, 'description': 'Giảm 30% , Ngộp Bank\n\n Toà Nhà Trần Hưng Đạo,p.Cô Giang, Q.1\n-Diện tích: 18x20. Công nhận 340m2. Dt Sàn 1.872m2\n- Hầm, Trệt, 8 Lầu, 2 Thang máy\n- HĐT: 700tr/tháng \n\n- Đi nhanh 300tỷ ,  Chạm Đáy 200 hơn'}


In [57]:
from pymongo import UpdateOne

processed = db[MONGODB_RPROCESSED_COLLECTION]

update_operations = []
for item in data_new:
    update_operations.append(
        UpdateOne(
            {"raw_id": item['raw_id']},
            {"$set": item}
        )
    )

processed.bulk_write(update_operations)

In [40]:
for item in data_new[:2]:
    print(item)

{'raw_id': '64f5ede00459b50fd4b6b9af', 'category': 'Nhà ở', 'region': 'Tp Hồ Chí Minh', 'ward': 'Phường Nguyễn Cư Trinh', 'list_time': 1693459941000, 'floors': 3, 'description': 'Nhà đường cống Quỳnh quận 1 dt 3.2*17 55m2 trệt lửng lầu 3ph ngủ nhà hẻm 3m 1/ chủ gấp bán  giải quyết công việc giá chào 7ty8 thương lượng cam kết nhà thật ko kê dưới mọi hình thức xem nhà 24/24'}
{'raw_id': '64f5ede00459b50fd4b6b9b0', 'category': 'Nhà ở', 'region': 'Tp Hồ Chí Minh', 'ward': 'Phường Cô Giang', 'list_time': 1693458004719}
